In [1]:
import pandas as pd
from prog.tools import *
from prog.db import *
import os, sqlalchemy
from tqdm import tqdm

In [2]:
root = os.getcwd()

clean_path = os.path.join(root, "data", "sort")        
os.makedirs(clean_path, exist_ok = True)
output_csv = os.path.join(clean_path, "data.csv")
output_json = os.path.join(clean_path, "output.json")        
waste_json = os.path.join(clean_path, "waste.json")

# 取得config
config_path = os.path.join(root, "prog", "config.json")
with open(config_path, 'r') as f:
    config = json.load(f)

In [3]:
db_con = config["database"]
db = Database(db_con)
host, port, user, password, database = db_con["host"], db_con["port"], db_con["user"], db_con["password"], db_con["database"]
con_info = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'
alchemy_conn = sqlalchemy.create_engine(con_info)

In [4]:
db_con = config["database"]
db = Database(db_con)

df = db.get_data(db_con["table"]["predict"])
remove = (df[["length", "width"]] <= 0).any(axis = 1) # 長寬為0無法切割
df = df[~remove].reset_index(drop = True)
print(f"shape = {df.shape}")
print(f"remove: {remove.sum()}")
df.head(3)

shape = (18526, 25)
remove: 8


,qrcode,order_id,transfer_order_id,item_id,cabinet,item_name,length,width,depth,color,...,width_cutting,qrcode_dis,diversion,number,is_disable,processing_code,create_by,create_time,modify_by,modify_time
0,BC11205013-0001,BC11205013,None,5,01_矮櫃,T腳,2600.0,130.0,18.0,BJ8321TX_18,...,130.0,BC1120501300011,None,1/1,N,None,admin,2023-07-12 13:50:02,admin,2023-07-12 13:50:02
1,BC11205013-0002,BC11205013,None,2,01_矮櫃,右側板,1320.0,560.0,18.0,BJ8321TX_18,...,560.0,BC1120501300022,500/160,1/1,N,(2),admin,2023-07-12 13:50:02,admin,2023-07-12 13:50:02
2,BC11205013-0003,BC11205013,None,1,01_矮櫃,左側板,1320.0,560.0,18.0,BJ8321TX_18,...,560.0,BC1120501300031,500/160,1/1,N,(2),admin,2023-07-12 13:50:02,admin,2023-07-12 13:50:02


df = df.sort_values("serial").reset_index(drop = True)
df = df.drop(["serial"], axis = 1)
df.head(3)

### Save data to database

db_con = config["db_connect"]["train"]
con_info = f'mysql+pymysql://{db_con["username"]}:{db_con["password"]}@{db_con["host"]}:{db_con["port"]}/{db_con["database"]}'
print(con_info)
conn = sqlalchemy.create_engine(con_info)

df.insert(0, "serial", df.index)
df.to_sql("a_chuanmai_sort_schdl_order_item", con = conn, if_exists = "replace", index = False)

### Sort

In [5]:
df = df.sort_values(["color", "order_id", "cabinet", "length", "width"], ascending = [True, True, True, False, False])
df = df.reset_index(drop = True)
df.head(3)

,qrcode,order_id,transfer_order_id,item_id,cabinet,item_name,length,width,depth,color,...,width_cutting,qrcode_dis,diversion,number,is_disable,processing_code,create_by,create_time,modify_by,modify_time
0,BC11205013-0005,BC11205013,None,6,01_矮櫃,背拉,443.0,96.0,18.0,*不拘_18,...,96.0,BC1120501300071,ABD,1/1,N,None,admin,2023-07-12 13:50:02,admin,2023-07-12 13:50:02
1,BS11202160-0033,BS11202160,None,18,01_矮櫃,背拉,964.0,96.0,18.0,*不拘_18,...,96.0,BS1120216000311,ABD,1/2,N,None,admin,2023-07-12 12:22:06,admin,2023-07-12 12:22:06
2,BS11202160-0034,BS11202160,None,18,01_矮櫃,背拉,964.0,96.0,18.0,*不拘_18,...,96.0,BS1120216000321,ABD,2/2,N,None,admin,2023-07-12 12:22:06,admin,2023-07-12 12:22:06


### Calculate area

In [6]:
df["area"] = df["length"] * df["width"] # 每個矩形的面積
length_limit = config["limit"]["length"]
width_limit = config["limit"]["width"]
area_limit = length_limit * width_limit
df["area_prob"] = df["area"] / area_limit # 矩形站箱子的面積
df.head(3)

,qrcode,order_id,transfer_order_id,item_id,cabinet,item_name,length,width,depth,color,...,diversion,number,is_disable,processing_code,create_by,create_time,modify_by,modify_time,area,area_prob
0,BC11205013-0005,BC11205013,None,6,01_矮櫃,背拉,443.0,96.0,18.0,*不拘_18,...,ABD,1/1,N,None,admin,2023-07-12 13:50:02,admin,2023-07-12 13:50:02,42528.0,0.004725
1,BS11202160-0033,BS11202160,None,18,01_矮櫃,背拉,964.0,96.0,18.0,*不拘_18,...,ABD,1/2,N,None,admin,2023-07-12 12:22:06,admin,2023-07-12 12:22:06,92544.0,0.010283
2,BS11202160-0034,BS11202160,None,18,01_矮櫃,背拉,964.0,96.0,18.0,*不拘_18,...,ABD,2/2,N,None,admin,2023-07-12 12:22:06,admin,2023-07-12 12:22:06,92544.0,0.010283


### Calculate waste and plate_id & Sort

In [7]:
df[["waste", "plate_id"]] = [1, 0]

start = 0 # 第n個箱子的第一個矩形於items中的index
end = 0 # 第n個箱子的最後一個矩形於items中的index
plate_id = 1 # 箱子id
accum_areas = 0 # 第n個箱子的累績使用面積

# 每次加入一塊矩形，並計算箱子的耗損率
pbar = tqdm(total = len(df), ncols=150)
while end < len(df):
    cut_items = df[["length", "width"]].to_numpy() # 轉換成newPacker可接收的格式
    rectangles = cut_items[start:end+1] # 欲放入箱子中的所有矩形
    length = cutting(length_limit, width_limit, rectangles) # 查看是否可全數放入箱子中，如果可放入，箱子中的矩形數量會加1
    accum_num = end - start + 1 # 目前測試放入箱中的矩形數量

    # 如果欲放入箱中的矩形數量和經過cutting後箱中的矩形數量不同，表示該矩形無法放入箱中，尋找其他可放入的矩形
    if (length != accum_num):
        init = df.iloc[end].copy()
        color, _, _ = df.loc[end, ["color", "length", "width"]]
        df1 = df.query("(color == @color) and (plate_id == 0)") # 抓出候選組合
        df1_index = df1[["length", "width"]].drop_duplicates(keep = "first").index # 只保留所有候選組合的第一個，節省搜索時間
        
        # 依序尋找所有可能的解
        for i, replace in enumerate(df1_index):
            pbar.set_postfix(plate_id = plate_id, waste = waste, test_num = f"{i+1}/{len(df1_index)}") # 更新目前搜索進度
            pbar.update(0)

            df.iloc[end] = df.iloc[replace]
            cut_items = df[["length", "width"]].to_numpy() # 轉換成newPacker可接收的格式
            rectangles = cut_items[start:end+1] # 欲放入箱子中的所有矩形
            length = cutting(length_limit, width_limit, rectangles) # 查看是否可全數放入箱子中，如果可放入，箱子中的矩形數量會加1
            
            # 有解則跳出迴圈
            if (length == accum_num):
                df.iloc[replace] = init
                break
        
        # 無可放入的矩形，新增一個箱子，並重設狀態
        if (length != accum_num):
            df.iloc[end] = init
            start = end
            plate_id += 1
            accum_areas = 0
            continue

    # 放入矩形進箱子裡
    accum_areas += cut_items[end][0] * cut_items[end][1] # 加入該矩形後，箱子的累績使用面積 (長 * 寬)
    waste = 1 - (accum_areas / (length_limit * width_limit)) # 箱子的耗損率
    # print(f"{start} ~ {end}, length = {length}, accum_num = {accum_num}, waste = {waste}, plate_id = {plate_id}")

    df.loc[end, "waste"] = waste
    df.loc[end, "plate_id"] = plate_id

    pbar.set_postfix(plate_id = plate_id, waste = waste) # 更新目前排序進度
    pbar.update(1)

    end += 1
    if (end < len(cut_items) - 1) and (df.loc[end, "color"] != df.loc[end - 1, "color"]): # 如果下一個矩形的顏色不同，則新增一個箱子，並重設狀態
        start = end
        plate_id += 1
        accum_areas = 0

pbar.close()

100%|████████████████████████████████████████████████████████████████████████████████| 18526/18526 [36:29<00:00,  8.46it/s, plate_id=824, waste=0.199]


### 耗損率

In [8]:
waste_score = calculate_mean_wast(df, waste_json)
waste_score["plate"]

{'waste': 0.0809, 'plate': 632, 'no_rm_waste': 0.2157, 'no_rm_plate': 824}

### Save

In [9]:
df = df.drop(["area", "area_prob", "waste", "plate_id"], axis = 1)
df.to_csv(output_csv, encoding='utf-8-sig', index = False)
df.head(3)

,qrcode,order_id,transfer_order_id,item_id,cabinet,item_name,length,width,depth,color,...,width_cutting,qrcode_dis,diversion,number,is_disable,processing_code,create_by,create_time,modify_by,modify_time
0,BC11205013-0005,BC11205013,None,6,01_矮櫃,背拉,443.0,96.0,18.0,*不拘_18,...,96.0,BC1120501300071,ABD,1/1,N,None,admin,2023-07-12 13:50:02,admin,2023-07-12 13:50:02
1,BS11202160-0033,BS11202160,None,18,01_矮櫃,背拉,964.0,96.0,18.0,*不拘_18,...,96.0,BS1120216000311,ABD,1/2,N,None,admin,2023-07-12 12:22:06,admin,2023-07-12 12:22:06
2,BS11202160-0034,BS11202160,None,18,01_矮櫃,背拉,964.0,96.0,18.0,*不拘_18,...,96.0,BS1120216000321,ABD,2/2,N,None,admin,2023-07-12 12:22:06,admin,2023-07-12 12:22:06
